In [88]:
import tweepy
import pandas as pd
import numpy as np

# tweepy 셋업 단계
consumer_key = 'YXLsJ8Fr2Ad3CxgW1hpJWYPjP'
consumer_secret = '3cnMDDcda8ndsJJuxuto9kBwv6Qzep2hFzxmoS41WYenXVw9tF'
access_token = '65360970-zXUkzrmRlVnYSTGRLwG40UD9DRTQaMOenc9ErzdZb'
access_token_secret = 'VbumqpNgmdVeIpRHwiPySPfTzq5koRwiuMgYNrnCCkCS2'
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)


def get_tweets(usernames):
    # 트위터에서 받은 law data 를 종류별로 담을 list들 생성
    user_name = [] 
    ID = []
    date = []
    time = []
    comment = []
    
    # join 함수를 써서 usernames에 한개한개 나눠있는 셀럽 이름들을 하나의 string으로 합치는 과정
    # " OR "는 셀럽 이름들을 하나의 스트링으로 합칠때 중간에 넣을 string, ex) ["@katyperry", "@justinbieber"] ==> "@katyperry OR @justinbieber"
    user = " OR ".join(usernames)
    
    # 정확한건 아니지만 tweepy.Cursor가 페이지 기능을 무효화 시키는것 같음, items()의 파라미터에 페이지 기능을 무시하고 검색출력할 아이템 갯수 입력
    # 첫번째 옵션 api.search는 검색 기능 사용하겠다는것이고, 두번째 q옵션은 검색창에 쓸 쿼리 내용
    # 처음 코딩때 데이터 제한이 걸린 이유는 검색 방법을 한 셀럽 이름 당 1000개의 item 검색 출력을 했기때문에.. 한명당 1000개씩 10명을 검색해서 제한걸림
    # 지금 코드는 한번 검색에 모든 셀럽 이름을 다 넣어서 검색함
    searched_tweets = [status for status in tweepy.Cursor(api.search,q=user, since="2019-08-17",\
                                                          until="2019-08-18",lang="en").items(100)] # 검색된 각 건들을 대괄호를 쳐주면서 바로 리스트형식 안에 넣어줌
    
    # 리스트에 넣어놓은 검색결과들을 for 문으로 한 건씩 처리
    for tweet in searched_tweets:
        # 앞서 만들어놓아던 list형식들에 분류해서 append로 넣음
        user_name.append(tweet.user.name) 
        ID.append("@" + tweet.user.screen_name) 
        comment.append(tweet.text.lower()) 

        time_split = str(tweet.created_at).split() # 시간을 날짜와 시간으로 나누는 코드
        date.append(time_split[0]) 
        time.append(time_split[1])
     
    # pandas 데이터 프레임으로 만들기 과정
    data = {} 
    data['user'] = user_name
    data['id'] = ID
    data['comment'] = comment
    data['date'] = date
    data['time'] = time
    df = pd.DataFrame(data) # pandas 데이터프레임으로 변환
    
    # numpy.select 함수를 이용해서 comment에 있는 내용중에 usernames에 있는 셀럽 아이디와 같은것을 발견하면 
    # 데이터프레임의 새로운 컬럼인 'keyword' 컬럼에 셀럽 아이디를 넣는 과정
    condition = [df['comment'].str.contains(i) for i in usernames]
    df['keyword'] =np.select(condition, usernames,"none")
    
    # keyword컬럼에서 none으로 표시된 row 제거하는 과정
    df.drop(df[df['keyword']=='none'].index, inplace=True)
    
    return df

usernames = ["@katyperry", "@justinbieber", "@barackobama", "@rihanna", \
             "@taylorswift13", "@ladygaga", "@theellenshow", \
             "@cristiano", "@jtimberlake", "@arianagrande"]

# 함수 실행
dataframe = get_tweets(usernames)



In [169]:
dataframe.head()

,comment,date,id,time,user,keyword
0,@chartdata @taylorswift13 queen https://t.co/o...,2019-08-17,@JefreeStar5,23:59:59,YassMamma (fan account),@taylorswift13
1,@mitchellvii who didn't have a world class gri...,2019-08-17,@TerniValentine,23:59:58,Allbizness,@barackobama
2,rt @barackobama: i’ve always been proud of wha...,2019-08-17,@ox_jakeox,23:59:58,JakeOx,@barackobama
3,rt @troyesivan: lover is a very special song b...,2019-08-17,@alansuricato,23:59:58,Lovercato 23.08,@taylorswift13
4,rt @vmas: happy anniversary 'sweetener'! 💛@ari...,2019-08-17,@picachusodi,23:59:56,Pica Chu,@arianagrande


In [98]:
# d 번 답

# 기존 데이터프레임 자료들을 이용해서 새로운 데이터 프레임 df_d 만드는 과정
ID = dataframe['keyword'].value_counts().index
mentioned = dataframe['keyword'].value_counts().values
df_d = pd.DataFrame(data={"ID": ID, "Mentioned": mentioned})

# numpy.select 함수를 사용해서 셀럽 아이디에 맞게 셀럽 풀네임을 새로운 컬럼 'Celebrity'에 입력하는 과정
condition = [df_d['ID'] == "@katyperry",df_d['ID'] == "@justinbieber", df_d['ID'] == "@barackobama", df_d['ID'] == "@rihanna",\
            df_d['ID'] == "@taylorswift13",df_d['ID'] == "@ladygaga",df_d['ID'] == "@theellenshow",df_d['ID'] == "@cristiano",\
            df_d['ID'] == "@jtimberlake",df_d['ID'] == "@arianagrande",]
returns = ["KATY PERRY", "Justin Bieber", "Barack Obama", "Rihanna", "Taylor Swift", "Lady Gaga", "Ellen DeGeneres",\
           "Cristiano Ronaldo", "Justin Timberlak", "Ariana Grande"]
df_d['Celebrity'] = np.select(condition, returns)
print(df_d)

,ID,Mentioned,Celebrity
0,@taylorswift13,45,Taylor Swift
1,@arianagrande,22,Ariana Grande
2,@barackobama,9,Barack Obama
3,@theellenshow,5,Ellen DeGeneres
4,@katyperry,4,KATY PERRY
5,@justinbieber,4,Justin Bieber
6,@ladygaga,4,Lady Gaga
7,@rihanna,2,Rihanna
8,@cristiano,1,Cristiano Ronaldo


In [176]:
# c번 답

# df_d로 부터 제일 mentioned된 셀럽 ID를 가져와서 기존의 데이터 프레임에서 그 ID가 포함된 데이터만 검색함
# 그 검색된 결과에서 'id' 컬럼에 있는 트윗 한 유저의 닉네임으로 count를 함
calls = dataframe[dataframe['keyword']==df_d['ID'][0]]['id'].value_counts().index[0]
count = dataframe[dataframe['keyword']==df_d['ID'][0]]['id'].value_counts().values[0]
print("{} mentioned {} the most with {} times from August 16 - 17, 2019.".format(calls, df_d['ID'][0], count))

@nmiller11 mentioned @taylorswift13 the most with 3 times from August 16 - 17, 2019.


In [142]:
# b번 답

# df_d로 부터 제일 많이 mentioned된 셀럽의 풀네임과 카운트 수를 format 함수를 이용해서 출력
print("{} has been tweeted the most with {} times.".format(df_d["Celebrity"][0],df_d["Mentioned"][0] ))

Taylor Swift has been tweeted the most with 45 times.


In [160]:
dic_name = {"@katyperry":"KATY PERRY","@justinbieber":"Justin Bieber", "@barackobama":"Barack Obama", \
            "@rihanna":"Rihanna","@taylorswift13":"Taylor Swift","@ladygaga":"Lady Gaga","@theellenshow":"Ellen DeGeneres",\
            "@cristiano":"Cristiano Ronaldo", "@jtimberlake":"Justin Timberlak" ,"@arianagrande":"Ariana Grande"}

In [177]:
dataframe[dataframe['keyword']=='@taylorswift13']['id'].value_counts()

@nmiller11          3
@TaylorSwizzly      2
@taybitxh           2
@boringkiana        2
@imfeelingTS7       1
@tookwancottage     1
@alansuricato       1
@hidalgocute        1
@taysfavgay         1
@LauraSwiftie23     1
@hollylaurenbihm    1
@_iNtanSuraiya      1
@shmellerz          1
@moireputation      1
@margotssharon      1
@loversafter        1
@miamisteven        1
@tannerstweetz      1
@R_Thobykov         1
@_TobyAaron         1
@X_1x1_2            1
@massone38          1
@fyevlln            1
@jordi_moo          1
@StellaHSCruz       1
@brentnerb          1
@ronlextall         1
@zweeran            1
@JefreeStar5        1
@dealicate_         1
@rep1989jess        1
@boylogia           1
@sugarprise         1
@yeritzacabrera     1
@vivibibby          1
@promschimie        1
@psychoforswift     1
@Lalisayaaah        1
@eisa65151810       1
@ethanbarberco      1
Name: id, dtype: int64

In [168]:
aaa = pd.DataFrame(dataframe.groupby("user")["keyword"].value_counts(sort=True))
print(aaa)
# aaa['user'][np.argmax(aaa['user'].values)]

                                                           keyword
user                                       keyword                
#WeAreWaitingJB5 💓💓                        @justinbieber         2
&                                          @taylorswift13        1
.                                          @arianagrande         1
A                                          @taylorswift13        1
Allbizness                                 @barackobama          1
Ariana Grande Access                       @arianagrande         1
Aroon Dani                                 @katyperry            1
Blbaeri                                    @cristiano            1
CryptoGaston                               @justinbieber         1
D.                                         @ladygaga             3
Darryl Buffkin💎                            @barackobama          1
EOD.Dave                                   @barackobama          1
Esmeralda                                  @taylorswift13     